# Structured hierarchical models

The goal of this project is to model essential genes for multiple conditions given that we have prior knowledge about functional relations of these genes. The model we want to use is:

\begin{align}
\mathbf{z} & \sim \text{MRF} \\
\tau^2 & \sim P(\cdot) \\
\mu_{z_g} & \sim \mathcal{N}(0, 1) \\
\gamma_g \mid z_g &  \sim \mathcal{N}(\mu_{z_g}, \tau^2) \\
\beta_{vg} \mid \gamma_g &  \sim \mathcal{N}(\gamma_g, \tau^2) \\
l_{vg} & \sim \text{Gamma}(1, 1) \\
\sigma_2 & \sim \text{Half-Cauchy}(0, 1) \\
x_{svg} \mid \beta_{vg} &  \sim \text{Pois}\left(l_{vg} \cdot \exp \left( \beta_{vg} \right), \sigma^2\right) \\
\end{align}


We start with some simple models, see them in the other notebooks.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
import pandas as pd
import pymc3 as pm
import numpy as np
import scipy as sp
import theano.tensor as tt

In [3]:
from sklearn.preprocessing import LabelEncoder
from pymc3 import  model_to_graphviz

In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import arviz as az

sns.set_style(
    "white", 
    {'xtick.bottom': True,
     'ytick.left': True,
     'axes.spines.top': False, 
     'axes.spines.right': False})

Since the MRF is particularly hard to estimate, we make use of a simpler representation first.

\begin{align}
\boldsymbol \pi & \sim \text{Dirichlet}(1, 1) \\
z_g & \sim \text{Categorical}(\boldsymbol \pi) \\
\tau_g^2 & \sim \text{Gamma}(1, 1) \\
\mu_{z_g} & \sim \mathcal{N}(0, 1) \\
\gamma_g \mid z_g &  \sim \mathcal{N}(\mu_{z_g}, \tau_g^2) \\
\tau_b^2 & \sim \text{Gamma}(1, 1) \\
\beta_{cg} \mid \gamma_g &  \sim \mathcal{N}(\gamma_g, \tau_b^2) \\
l_{cgs} & \sim \text{Lognormal}(1, 1) \\
x_{cgs} \mid \beta_{vg} &  \sim \text{Pois}\left(l_{vgs} \cdot \exp \left( \beta_{cg} \right) \right) \\
\end{align}

## Data

Since it is always _both_ replicates that have a low read count, it is safe to assume that these are no batch effects.

In [6]:
infile = "/home/simon/PROJECTS/cell_lines/src/shm/data_raw/read_counts-normalized.tsv"

In [105]:
def _load_data(infile):    
    dat = pd.read_csv(infile, sep="\t")
    dat = dat.query("Gene == 'POLR3K' | Gene == 'BCR'")    
    dat2 = dat.copy()
    dat2["Condition"] = "Penis"
    dat = dat.append(dat2)
    dat = (dat[["Condition", "Gene", "sgRNA", "r1", "r2"]]
           .query("Gene != 'Control'")
           .melt(id_vars=["Gene", "Condition", "sgRNA"],
                 value_vars=["r1", "r2"],
                 var_name="replicate",
                 value_name="counts")
           .sort_values(["Gene", "Condition", "sgRNA", "replicate"])
    )
    dat["sgRNA"] = LabelEncoder().fit_transform(dat["sgRNA"].values)
    dat["counts"] = sp.floor(dat["counts"].values)
    return dat

In [106]:
read_counts = _load_data(infile)

In [107]:
read_counts

,Gene,Condition,sgRNA,replicate,counts
0,BCR,Neuron,0,r1,6538.0
40,BCR,Neuron,0,r2,6535.0
1,BCR,Neuron,1,r1,11176.0
41,BCR,Neuron,1,r2,11051.0
2,BCR,Neuron,2,r1,21733.0
42,BCR,Neuron,2,r2,21529.0
3,BCR,Neuron,3,r1,9350.0
43,BCR,Neuron,3,r2,9358.0
4,BCR,Neuron,4,r1,12643.0
44,BCR,Neuron,4,r2,12467.0


## Inference

In [141]:
n, _ = read_counts.shape
le = LabelEncoder()

gene_idx = le.fit_transform(read_counts["Gene"].values)
con_idx = le.fit_transform(read_counts["Condition"].values)

len_genes = len(sp.unique(gene_idx))
len_conditions = len(sp.unique(con_idx))
len_sirnas = len(sp.unique(read_counts["sgRNA"].values))
len_replicates = len(sp.unique(read_counts["replicate"].values))
len_sirnas_per_gene = int(len_sirnas / len_genes)

beta_idx = np.tile(range(len_genes), len_conditions)
beta_data_idx = np.repeat(beta_idx, int(n / len(beta_idx)))

l_idx = np.repeat(range(len_sirnas * 2), len_replicates )

In [148]:
with pm.Model() as model:
    p = pm.Dirichlet("p", a=np.array([1.0, 1.0]), shape=2)
    pm.Potential("p_pot", tt.switch(tt.min(p) < 0.05, -np.inf, 0))
    category = pm.Categorical("category", p=p, shape=len_genes)

    tau_g = pm.Gamma("tau_g", 1.0, 1.0, shape=1)
    mean_g = pm.Normal("mu_g", mu=np.array([0, 0]), sd=0.5, shape=2)
    pm.Potential("m_opot", tt.switch(mean_g[1] - mean_g[0] < 0, -np.inf, 0))
    gamma = pm.Normal("gamma", mean_g[category], tau_g, shape=len_genes)

    tau_b = pm.Gamma("tau_b", 1.0, 1.0, shape=1)
    if len_conditions == 1:
        beta = pm.Deterministic("beta", gamma)
    else:
        beta_non = pm.Normal('beta_non', 0, 1, shape= len(beta_idx))
        beta = pm.Deterministic("beta", gamma[beta_idx] + tau_b * beta_non)
    l = pm.Lognormal("l", 0, 0.25, shape=len_sirnas * 2)

    pm.Poisson(
      "x",
      mu =       np.exp(beta[beta_data_idx]) * l[l_idx],
      observed = sp.squeeze(read_counts["counts"].values),
    )

In [150]:
mod = model_to_graphviz(model)

In [151]:
mod.render("model.dot")

'model.dot.pdf'